# Introduction

This project is designed to be fully dynamic. You can mix and match
multiple LLM providers and adjust the behavior of your agent with a
small amount of structured input.

## Supported LLM Providers

-   **Ollama**
-   **OpenAI**

## Supported Agent Roles

Your agent can act in many different roles, for example: - attendant\
- seller\
- guide\
- professor\
- and more...

## Required Inputs

To configure your agent, you simply need to provide: - **You are:**
\_\_\_\_\_\_\
- **Your purpose is:** \_\_\_\_\_\_\
- **You need to avoid:** \_\_\_\_\_\_\
- **You need to be:** \_\_\_\_\_\_\
- **Values / Principles:** \_\_\_\_\_\_

## Adding Context

You can enrich the agent with additional context using files such as: -
JSON\
- CSV\
- XML

Just make sure the structure of each file is valid.

## Install dependencies

In [1]:
!pip install --upgrade langchain langchain-community langchain-openai langchain-ollama
!pip install pandas xmltodict
!pip install ipywidgets==8.1.2

## Select the LLM (Ollama or OpenAI)

In [23]:
import ipywidgets as widgets
from IPython.display import display

llm_choice = widgets.Dropdown(
    options=['OpenAI', 'Ollama'],
    value='OpenAI',
    description='LLM Provider:',
)

display(llm_choice)

Dropdown(description='LLM Provider:', options=('OpenAI', 'Ollama'), value='OpenAI')

## Initialize the LLM with LangChain

In [25]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

def get_llm():
    if llm_choice.value == "OpenAI":
        return ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)
    else:
        return ChatOllama(model="llama3.2", temperature=0.2)

llm = get_llm()

## User-defined persona block



In [5]:
you_are = widgets.Text(description='You are:')
purpose = widgets.Text(description='Purpose:')
avoid = widgets.Text(description='Avoid:')
be_need = widgets.Text(description='Be:')
values = widgets.Textarea(description='Values:')

display(you_are, purpose, avoid, be_need, values)

Text(value='', description='You are:')

Text(value='', description='Purpose:')

Text(value='', description='Avoid:')

Text(value='', description='Be:')

Textarea(value='', description='Values:')

In [32]:
print(you_are.value)
print(purpose.value)
print(avoid.value)
print(be_need.value)
print(values.value)

an English Teacher
support ABC clients
slangs
kind and friendly
honesty


## Context loading via LangChain loaders

In [33]:
import ipywidgets as widgets
from IPython.display import display

uploader = widgets.FileUpload(
    accept='',        # accepts all file types
    multiple=True     # allow multiple files
)

display(uploader)

FileUpload(value=(), description='Upload', multiple=True)

In [9]:
uploaded = uploader.value

Parse file:

In [13]:
context_text = ""

for filename, fileinfo in uploaded.items():
    content = fileinfo['content'].decode()

    if filename.endswith(".json"):
        context_text = json.dumps(json.loads(content), indent=2)

    elif filename.endswith(".csv"):
        import pandas as pd
        from io import StringIO
        df = pd.read_csv(StringIO(content))
        context_text = df.to_markdown()

    elif filename.endswith(".xml"):
        import xmltodict
        context_text = json.dumps(xmltodict.parse(content), indent=2)

    else:
        context_text = content

AttributeError: 'tuple' object has no attribute 'items'

## Create the LangChain System Prompt

In [34]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are {you_are}.
Your purpose is {purpose}.
You must avoid: {avoid}.
You must be: {be_need}.

Values and Principles:
{values}

--- CONTEXT FROM FILES ---
{context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

## Create a chain

In [35]:
from langchain_core.messages import SystemMessage, HumanMessage

def build_chain(system_prompt):
    return [
        SystemMessage(content=system_prompt),
        HumanMessage(content="{input}")
    ] | get_llm()


## own simple memory

In [36]:
class SimpleMemory:
    def __init__(self):
        self.history = []

    def add(self, user_input, model_output):
        self.history.append(("user", user_input))
        self.history.append(("assistant", model_output))

    def get_messages(self):
        return [
            {"role": role, "content": content}
            for role, content in self.history
        ]

If you are using ollama, certfied to run it locally

In [37]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

memory = SimpleMemory()

def run_agent(user_input):
    history_msgs = memory.get_messages()

    # Build final message list
    messages = [
        {"role": "system", "content": system_prompt},
        *history_msgs,
        {"role": "user", "content": user_input}
    ]

    result = get_llm().invoke(messages)
    memory.add(user_input, result.content)

    return result.content

User Input:

In [38]:
user_input = widgets.Text(description='Welcome! How can I support you?')
display(user_input)

Text(value='', description='Welcome! How can I support you?')

In [39]:
print(user_input.value)

Can you support me in English Grammar?


In [40]:
run_agent(user_input.value)

"I'd be happy to help you with English grammar. What specific area of grammar would you like help with? Do you have a particular question or topic in mind, such as:\n\n* Verb tenses and conjugations\n* Sentence structure and clauses\n* Modality and conditionals\n* Punctuation and commas\n* Something else?\n\nLet me know and I'll do my best to assist you.\n\nAlso, please let me know what level of English grammar you are at. Are you a beginner, intermediate, or advanced learner? This will help me tailor my responses to your needs."